In [20]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [21]:
def load_weather_data(region):
    for year in range(2010, 2025):
        if year == 2010:
            df = pd.read_csv(f'{region}/{year} {region} 날씨 데이터.csv', encoding='euc-kr')
        else:
            df = pd.concat([df, pd.read_csv(f'{region}/{year} {region} 날씨 데이터.csv', encoding='euc-kr')])
        
    df = df[['지점명','일시', '평균기온(°C)', '평균 상대습도(%)', '합계 일사량(MJ/m2)']]
    df = df.rename(columns={'지점명': '지점', '일시': '일시', '평균기온(°C)': '평균기온', '평균 상대습도(%)': '평균습도', '합계 일사량(MJ/m2)': '일평균일사량'})
    df['일평균일사량'] = (df['일평균일사량'] / 24).round(1)

    # 일평균일사량의 결측치를 선형 보간법으로 채우는 방법 사용
    df['일평균일사량'] = df['일평균일사량'].interpolate(method='linear', limit_direction='both')
    df['평균기온'] = df['평균기온'].interpolate(method='linear', limit_direction='both')
    df['평균습도'] = df['평균습도'].interpolate(method='linear', limit_direction='both')
    df['일시'] = pd.to_datetime(df['일시'])
    
    return df

In [22]:
def extract_weather_data(date, weather_df):
    # 특정 날짜 기준 3개월 전 날짜
    start_date = date - pd.DateOffset(months=3)
    # 특정 날짜 기준 1개월 후 날짜
    end_date = date + pd.DateOffset(months=1)
    
    # 날짜 범위에 해당하는 데이터 추출
    mask = (weather_df['일시'] >= start_date) & (weather_df['일시'] <= end_date)
    filtered_df = weather_df.loc[mask]
    
    return filtered_df

In [23]:
region_kr =['강원도', '경기도', '경상남도', '경상북도', '광주시', '대구시', '대전시', '부산시', '서울시', 
            '울산시', '인천시', '전라남도', '전라북도', '제주도', '충청북도', '충청남도']
region_en = ['gangwon', 'gyeonggi', 'gyeongnam', 'gyeongbuk', 'gwangju', 'daegu', 'daejeon', 'busan', 'seoul', 
            'ulsan', 'incheon', 'jeonnam', 'jeonbuk', 'jeju', 'chungbuk', 'chungnam']

In [24]:
for region_index in range(len(region_kr)):
    danpung_df = pd.read_excel(f'data/{region_kr[region_index]} 단풍나무 데이터.xlsx')
    weather_df = load_weather_data(region_kr[region_index])

    danpung_df['단풍시작'] = pd.to_datetime(danpung_df['단풍시작'], errors='coerce')
    danpung_df['단풍절정'] = pd.to_datetime(danpung_df['단풍절정'], errors='coerce')
    danpung_df = danpung_df.dropna(subset=['단풍시작', '단풍절정']).reset_index(drop=True)

    # 모든 단풍 시작일에 대해 데이터 추출
    all_weather_data = pd.DataFrame()

    for start_date in danpung_df['단풍시작']:
        temp_df = extract_weather_data(start_date, weather_df)
        all_weather_data = pd.concat([all_weather_data, temp_df])

    # 중복 제거 및 날짜순 정렬
    all_weather_data = all_weather_data.drop_duplicates().sort_values('일시')

    print(f"전체 추출된 데이터 기간: {all_weather_data['일시'].min()} ~ {all_weather_data['일시'].max()}")
    print(f"전체 데이터 수: {len(all_weather_data)}")

    danpung_merged_df = pd.merge(all_weather_data, danpung_df[['지점', '년도', '단풍절정']], left_on=['지점', all_weather_data['일시'].dt.year.astype(int)],
                                right_on=['지점', '년도'], 
                                how='inner')
    
    # 특성과 목표 변수 설정
    X = danpung_merged_df[['평균기온', '평균습도', '일평균일사량']]
    # 날짜를 연중 일수로 변환
    y = danpung_merged_df['단풍절정'].dt.dayofyear

    # 특성 스케일링
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 데이터를 훈련 세트와 테스트 세트로 분할
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=42)

    # 예측 모델 만들기
    model = RandomForestRegressor(n_estimators=10, max_depth=10)
    model.fit(X_train, y_train)

    # 모델 예측
    y_pred = model.predict(X_test)

    # 모델 정확도 평가
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    print(f"{region_kr[region_index]} 단풍나무 모델 정확도 (RMSE): {rmse:.2f}")

    pickle.dump(model, open(f'model/{region_en[region_index]}_maple_model.pkl', 'wb'))

전체 추출된 데이터 기간: 2010-07-12 00:00:00 ~ 2024-11-21 00:00:00
전체 데이터 수: 26034
강원도 단풍나무 모델 정확도 (RMSE): 5.24
전체 추출된 데이터 기간: 2010-07-19 00:00:00 ~ 2024-11-30 00:00:00
전체 데이터 수: 9476
경기도 단풍나무 모델 정확도 (RMSE): 4.90
전체 추출된 데이터 기간: 2010-07-25 00:00:00 ~ 2024-12-02 00:00:00
전체 데이터 수: 25243
경상남도 단풍나무 모델 정확도 (RMSE): 6.29
전체 추출된 데이터 기간: 2010-07-14 00:00:00 ~ 2024-12-02 00:00:00
전체 데이터 수: 28208
경상북도 단풍나무 모델 정확도 (RMSE): 5.66
전체 추출된 데이터 기간: 2010-07-28 00:00:00 ~ 2024-12-02 00:00:00
전체 데이터 수: 1843
광주시 단풍나무 모델 정확도 (RMSE): 6.73
전체 추출된 데이터 기간: 2010-08-01 00:00:00 ~ 2024-12-05 00:00:00
전체 데이터 수: 2039
대구시 단풍나무 모델 정확도 (RMSE): 6.74
전체 추출된 데이터 기간: 2010-07-23 00:00:00 ~ 2024-12-06 00:00:00
전체 데이터 수: 1856
대전시 단풍나무 모델 정확도 (RMSE): 6.65
전체 추출된 데이터 기간: 2010-07-30 00:00:00 ~ 2023-12-07 00:00:00
전체 데이터 수: 1847
부산시 단풍나무 모델 정확도 (RMSE): 7.76
전체 추출된 데이터 기간: 2010-07-26 00:00:00 ~ 2024-12-04 00:00:00
전체 데이터 수: 1858
서울시 단풍나무 모델 정확도 (RMSE): 4.47
전체 추출된 데이터 기간: 2010-07-29 00:00:00 ~ 2024-12-05 00:00:00
전체 데이터 수: 1857
울산시 단풍나무 모델 정확

In [25]:
for region_index in range(len(region_kr)):
    beojkkoch_df = pd.read_excel(f'data/{region_kr[region_index]} 벚나무 데이터.xlsx') 
    weather_df = load_weather_data(region_kr[region_index])

    beojkkoch_df['개화'] = pd.to_datetime(beojkkoch_df['개화'], errors='coerce')
    beojkkoch_df = beojkkoch_df.dropna(subset=['개화']).reset_index(drop=True)

    # 모든 개화일에 대해 데이터 추출
    all_weather_data = pd.DataFrame()

    for bloom_date in beojkkoch_df['개화']:
        temp_df = extract_weather_data(bloom_date, weather_df)
        all_weather_data = pd.concat([all_weather_data, temp_df])

    # 중복 제거 및 날짜순 정렬
    all_weather_data = all_weather_data.drop_duplicates().sort_values('일시')

    print(f"전체 추출된 데이터 기간: {all_weather_data['일시'].min()} ~ {all_weather_data['일시'].max()}")
    print(f"전체 데이터 수: {len(all_weather_data)}")

    beojkkoch_merged_df = pd.merge(all_weather_data, beojkkoch_df[['지점', '년도', '개화']], left_on=['지점', all_weather_data['일시'].dt.year.astype(int)],
                                right_on=['지점', '년도'], 
                                how='inner')
    
    # 특성과 목표 변수 설정
    X = beojkkoch_merged_df[['평균기온', '평균습도', '일평균일사량']]
    # 날짜를 연중 일수로 변환
    y = beojkkoch_merged_df['개화'].dt.dayofyear

    # 특성 스케일링
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 데이터를 훈련 세트와 테스트 세트로 분할
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=42)

    # 예측 모델 만들기
    model = RandomForestRegressor(n_estimators=10, max_depth=10)
    model.fit(X_train, y_train)

    # 모델 예측
    y_pred = model.predict(X_test)

    # 모델 정확도 평가
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    print(f"{region_kr[region_index]} 벚나무 모델 정확도 (RMSE): {rmse:.2f}")

    pickle.dump(model, open(f'model/{region_en[region_index]}_blossom_model.pkl', 'wb'))

전체 추출된 데이터 기간: 2010-01-14 00:00:00 ~ 2024-05-05 00:00:00
전체 데이터 수: 26434
강원도 벚나무 모델 정확도 (RMSE): 9.40
전체 추출된 데이터 기간: 2010-01-12 00:00:00 ~ 2024-05-03 00:00:00
전체 데이터 수: 9410
경기도 벚나무 모델 정확도 (RMSE): 5.99
전체 추출된 데이터 기간: 2010-01-01 00:00:00 ~ 2024-04-23 00:00:00
전체 데이터 수: 25991
경상남도 벚나무 모델 정확도 (RMSE): 4.38
전체 추출된 데이터 기간: 2010-01-03 00:00:00 ~ 2024-05-01 00:00:00
전체 데이터 수: 26981
경상북도 벚나무 모델 정확도 (RMSE): 6.10
전체 추출된 데이터 기간: 2010-01-01 00:00:00 ~ 2024-04-27 00:00:00
전체 데이터 수: 1829
광주시 벚나무 모델 정확도 (RMSE): 5.47
전체 추출된 데이터 기간: 2010-01-05 00:00:00 ~ 2024-04-27 00:00:00
전체 데이터 수: 2075
대구시 벚나무 모델 정확도 (RMSE): 4.54
전체 추출된 데이터 기간: 2010-01-16 00:00:00 ~ 2024-04-30 00:00:00
전체 데이터 수: 1826
대전시 벚나무 모델 정확도 (RMSE): 6.97
전체 추출된 데이터 기간: 2010-01-01 00:00:00 ~ 2024-04-25 00:00:00
전체 데이터 수: 2052
부산시 벚나무 모델 정확도 (RMSE): 4.82
전체 추출된 데이터 기간: 2010-01-12 00:00:00 ~ 2024-05-01 00:00:00
전체 데이터 수: 1823
서울시 벚나무 모델 정확도 (RMSE): 7.06
전체 추출된 데이터 기간: 2010-01-03 00:00:00 ~ 2024-04-28 00:00:00
전체 데이터 수: 1831
울산시 벚나무 모델 정확도 (RMSE): 

In [35]:
# 가상의 데이터로 예측하기
model = pickle.load(open('model/seoul_blossom_model.pkl', 'rb'))

# 가상의 데이터 생성
virtual_data = [20, 30, 50]  # 평균기온, 평균습도, 일평균일사량 (float로 변경)
virtual_data_scaled = scaler.transform([virtual_data])  # 스케일링 적용

# 예측 수행
predicted_bloom_day = model.predict(virtual_data_scaled)
print(f"예측된 벚꽃 개화일: {predicted_bloom_day[0]:.2f}일")

# 예측된 벚꽃 개화일을 월일로 변환
predicted_bloom_date = pd.to_datetime(predicted_bloom_day[0], unit='D', origin=pd.Timestamp('2024-01-01'))
print(f"예측된 벚꽃 개화일: {predicted_bloom_date.strftime('%Y-%m-%d')}")

예측된 벚꽃 개화일: 95.60일
예측된 벚꽃 개화일: 2024-04-05


/opt/homebrew/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
